In [1]:
import torch
import torch.nn as nn
import torchvision 
import pandas as pd
import numpy as np



In [5]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
data_train = pd.read_csv('dataAll.csv')
data_train.head()
data_train_add = data_train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = data_train.iloc[:,30:]
data_train_add.to_csv("./processed_data/train_pro.csv",index = False)
m = data_train_add.describe()
m.to_csv("./processed_data/des_pro.csv",index = False)

train_loc_list=[]
for i in range(len(train_loc)):
    train_loc_list.append([train_loc.iloc[i,0],train_loc.iloc[i,1]])
    
def cos_vali(data_list,class_list,test_size = 0.2):
    data_class_list = list(zip(data_list, class_list))
    import random
    random.shuffle(data_class_list)
    index = int(len(data_class_list)*test_size)#+1
    train_list = data_class_list[index:]
    test_list = data_class_list[:index]
    train_data_list, train_class_list = zip(*train_list)
    test_data_list, test_class_list = zip(*test_list)
    return list(train_data_list),list(test_data_list),list(train_class_list),list(test_class_list)
X_train, X_test, y_train, y_test = cos_vali(np.array(data_train_add).tolist(),train_loc_list)
train_data_list, test_data_list, train_class_list, test_class_list = cos_vali(np.array(data_train_add).tolist(),train_loc_list)

In [9]:
print('train_data_list:{}'.format(len(train_data_list)))
print('test_data_list:{}'.format(len(test_data_list)))

train_data_list:3091
test_data_list:772


In [15]:

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

input_size = 30
output_size = 2

num_epochs = 100
learning_rate = 0.001

x_train, x_test, y_train, y_test = np.array(X_train),np.array(X_test),np.array(y_train),np.array(y_test)


model = nn.Linear(input_size,output_size)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(x_train)
    targets = torch.from_numpy(y_train)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Plot the graph
predicted = model(torch.from_numpy(x_train)).detach().numpy()
plt.plot(x_train, y_train, 'ro', label='Original data')
plt.plot(x_train, predicted, label='Fitted line')
plt.legend()
plt.show()

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'